In [ ]:
%matplotlib inline

import cv2
import pycolmap
import poselib
import numpy as np
import os
from matplotlib import pyplot as plt
from typing import List, Dict, Tuple
import numpy.typing as npt
import json
import sys
from pathlib import Path

import visualizations as viz
from nptyping import NDArray, Int, Shape, Float16

datasetFolder =  os.path.join("..", "data", "captured_2022_08_27_19_28")

In [ ]:
def plotCVImage(image: cv2.Mat):
    plt.figure(figsize=(80, 8))
    plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    # as opencv loads in BGR format by default, we want to show it in RGB.
    plt.show()
    plt.tight_layout()



def visualizeMatches(img1, keyp1, img2, keyp2, matches):
    img1_color = cv2.cvtColor(img1, cv2.COLOR_GRAY2BGR)
    img2_color = cv2.cvtColor(img2, cv2.COLOR_GRAY2BGR)

    draw_params = dict(
        # matchColor=(0, 255, 0),
        # singlePointColor=(255, 0, 0),
        flags=cv2.DrawMatchesFlags_DEFAULT,
    )
    img3 = cv2.drawMatches(
        img1_color, keyp1, img2_color, keyp2, matches, None, **draw_params
    )
    plotCVImage(img3)



def PlotCameras(
    camsR: List[NDArray[Shape["3, 3"], Float16]],
    camsTrans: List[NDArray[Shape["3, 1"], Float16]],
):
    fig = plt.figure(figsize=(80, 8))
    ax = fig.add_subplot(111, projection="3d")
    ax.set_xlabel("X")
    ax.set_ylabel("Y")
    ax.set_zlabel("Z")
    for R, t in zip(camsR, camsTrans):
        viz.PlotCamera(R, t.squeeze(), ax)


def plotCVImagesHorizontal(images, addHorizontalLines = False):
    h_img = cv2.hconcat(images)
    h_img = cv2.cvtColor(h_img, cv2.COLOR_BGR2RGB)
    if addHorizontalLines:
        colors = [(255,0,0), (0,255,0), (255, 165, 0)]

        height = h_img.shape[0]
        width = h_img.shape[1]
        for i, y in enumerate(range(0, height, int(np.floor(height / 20)))):
            cv2.line(h_img, (0, y), (width, y), colors[i % len(colors)], thickness=2)

    fig, ax = plt.subplots(figsize=(180, 20))
    ax.imshow(h_img, interpolation='bilinear')
    plt.tight_layout()
    # plt.imshow(h_img)
    # plt.figure(figsize=(160, 80))
    # # as opencv loads in BGR format by default, we want to show it in RGB.
    # plt.show()
    # plt.tight_layout()


In [ ]:
# from pypopsift import popsift

# # causes segfault do not use
# def extractPopSiftFeatures(filename: os.path):
#     config = {
#         "sift_peak_threshold": 0.1,
#         "sift_edge_threshold": 10.0,
#         "feature_min_frames": 8000,
#         "feature_use_adaptive_suppression": False,
#     }

#     image = cv2.imread(filename, cv2.IMREAD_GRAYSCALE)

#     if image is None:
#         raise IOError("Unable to load image {}".format(filename))

#     print(image.shape)

#     points, desc = popsift(
#         image.astype(np.uint8),  # values between 0, 1
#         peak_threshold=config["sift_peak_threshold"],
#         edge_threshold=config["sift_edge_threshold"],
#         target_num_features=config["feature_min_frames"],
#     )

#     print(points.shape)
#     print(points)
#     print(desc.shape)
#     print(desc)


# extractPopSiftFeatures(os.path.join(datasetFolder, "1604_left.png"))


In [ ]:
def extractCVSiftFeatures(
    img: cv2.Mat,
) -> Tuple[List[cv2.KeyPoint], npt.NDArray[np.float64]]:
    sift = cv2.SIFT_create()

    keypoints, desc = sift.detectAndCompute(img, None)

    img_color = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
    plotCVImage(
        cv2.drawKeypoints(
            img_color,
            keypoints,
            None,
            (255, 0, 255),
            flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS,
        )
    )
    return keypoints, desc


img1 = cv2.imread(
    os.path.join(datasetFolder, "1604_left.png"),
    cv2.IMREAD_GRAYSCALE,
)

features = extractCVSiftFeatures(img1)


In [ ]:
from itertools import compress


def matchSurfFlann(
    desc1: npt.NDArray[np.float64], desc2: npt.NDArray[np.float64]
) -> List[cv2.DMatch]:

    # FLANN parameters
    FLANN_INDEX_KDTREE = 1
    index_params = dict(algorithm=FLANN_INDEX_KDTREE, trees=5)
    search_params = dict(checks=50)  # or pass empty dictionary
    flann = cv2.FlannBasedMatcher(index_params, search_params)
    matches = flann.knnMatch(desc1, desc2, k=2)
    # Need to draw only good matches, so create a mask
    matchesMask = [False for i in range(len(matches))]
    # ratio test as per Lowe's paper
    for i, (m, n) in enumerate(matches):
        if m.distance < 0.7 * n.distance:
            matchesMask[i] = True

    matches = map(lambda match: match[0], matches)
    return list(compress(matches, matchesMask))


img1 = cv2.imread(
    os.path.join(datasetFolder, "1604_left.png"),
    cv2.IMREAD_GRAYSCALE,
)

img2 = cv2.imread(
    os.path.join(datasetFolder, "1604_right.png"),
    cv2.IMREAD_GRAYSCALE,
)

kp1, desc1 = extractCVSiftFeatures(img1)

kp2, desc2 = extractCVSiftFeatures(img2)

matches = matchSurfFlann(desc1, desc2)
print("matches:", len(matches))
visualizeMatches(img1, kp1, img2, kp2, matches)


In [ ]:
with open(os.path.join(datasetFolder, "calibration.json")) as json_file:
    calibrationCam = json.load(json_file)

calibrationLeft = calibrationCam["cameraData"][0][1]
calibrationRight = calibrationCam["cameraData"][1][1]
rectifiedLeftHomography = np.array(
    calibrationCam["stereoRectificationData"]["rectifiedRotationLeft"]
)
rectifiedRightHomography = np.array(
    calibrationCam["stereoRectificationData"]["rectifiedRotationRight"]
)

print(calibrationLeft)
print(calibrationRight)
print(rectifiedLeftHomography)
print(rectifiedRightHomography)


def extractPoseLibCameraParams(calibration: Dict):
    intrinsics = {
        "model": "OPENCV",
        "width": calibration["width"],
        "height": calibration["height"],
        "params": [
            calibration["intrinsicMatrix"][0][0],
            calibration["intrinsicMatrix"][1][1],
            calibration["intrinsicMatrix"][0][2],
            calibration["intrinsicMatrix"][1][2],
            *calibration["distortionCoeff"][0:4],
        ],
    }
    return intrinsics


cameraLeft = extractPoseLibCameraParams(calibrationLeft)
print(cameraLeft)

cameraRight = extractPoseLibCameraParams(calibrationRight)
print(cameraRight)

rightToLeftGtRot = np.array(calibrationRight["extrinsics"]["rotationMatrix"])

origTranslation = calibrationRight["extrinsics"]["translation"]
rightToLeftGtTrans = np.array(
    [origTranslation["x"], origTranslation["y"], origTranslation["z"]]
)

intrinsicsLeft = np.array(calibrationLeft["intrinsicMatrix"])
intrinsicsRight = np.array(calibrationRight["intrinsicMatrix"])
distortionLeft = np.array(calibrationLeft["distortionCoeff"])
distortionRight = np.array(calibrationRight["distortionCoeff"])


In [ ]:
def extract2DMatches(matches, kp1, kp2):
    img1Pts = [np.array(kp1[mat.queryIdx].pt).reshape(2, 1) for mat in matches]
    img2Pts = [np.array(kp2[mat.trainIdx].pt).reshape(2, 1) for mat in matches]
    return img1Pts, img2Pts


img1Pts, img2Pts = extract2DMatches(matches, kp1, kp2)


In [ ]:
help(poselib.RansacOptions)
relativePose: poselib.CameraPose = poselib.CameraPose()
relativePose.R = rightToLeftGtRot
relativePose.t = rightToLeftGtTrans

print(relativePose)
relativePoseUpdated = relativePose

ransacParams = poselib.RansacOptions()
ransacParams["progressive_sampling"] = True
ransacParams["max_reproj_error"] = 1
print(ransacParams)

bundleParams = poselib.BundleOptions()
bundleParams["verbose"] = True
bundleParams["max_iterations"] = 0
print(bundleParams)

relativePoseRes = poselib.estimate_relative_pose(
    img2Pts, img1Pts, cameraRight, cameraLeft, ransacParams, bundleParams
)
# print(relativePoseRes)
relativePoseUpdated = relativePoseRes[0]
print(relativePoseUpdated)

inliers: List[bool] = relativePoseRes[1]["inliers"]
inlierMatches = list(compress(matches, inliers))
visualizeMatches(img1, kp1, img2, kp2, inlierMatches)


relPoseR = relativePoseUpdated.R
relPoseTr = relativePoseUpdated.t


In [ ]:
relPoseR = rightToLeftGtRot
relPoseTr = rightToLeftGtTrans

In [ ]:
def extractROI(image, roi):
    x, y, w, h = roi
    return image[y : y + h, x : x + w]


def rectifyStereo(
    imgLeft,
    imgRight,
    intrinsicsLeft,
    distortionLeft,
    intrinsicsRight,
    distortionRight,
    rightToLeftR,
    rightToLeftTrans,
):

    imgShape = imgLeft.shape[::-1]
    print(imgShape)
    rectifyRes = cv2.stereoRectify(
        intrinsicsLeft,
        distortionLeft,
        intrinsicsRight,
        distortionRight,
        imgShape,
        rightToLeftR,
        rightToLeftTrans,
    )

    Rleft, Rright, Pleft, Pright, Q, roiLeft, roiRight = rectifyRes
    print(roiLeft)
    print(roiRight)
    mapxLeft, mapyLeft = cv2.initUndistortRectifyMap(
        intrinsicsLeft, distortionLeft, Rleft, Pleft, imgShape, cv2.CV_32FC1
    )

    mapxRight, mapyRight = cv2.initUndistortRectifyMap(
        intrinsicsRight, distortionRight, Rright, Pright, imgShape, cv2.CV_32FC1
    )

    remapedLeft = cv2.remap(
        imgLeft, mapxLeft, mapyLeft, cv2.INTER_LINEAR, cv2.BORDER_CONSTANT
    )
    remapedRight = cv2.remap(
        imgRight, mapxRight, mapyRight, cv2.INTER_LINEAR, cv2.BORDER_CONSTANT
    )

    return remapedLeft, remapedRight


print(relPoseR)
print(relPoseTr)

remapedLeft, remapedRight = rectifyStereo(
    img1,
    img2,
    intrinsicsLeft,
    distortionLeft,
    intrinsicsRight,
    distortionRight,
    relPoseR,
    relPoseTr,
)



In [ ]:
plotCVImagesHorizontal([remapedLeft, remapedRight], True)
PlotCameras([np.identity(3), relPoseR], [np.zeros((3)), relPoseTr])
# PlotCameras([relPoseR], [relPoseTr])
